In [264]:
from ris import pysqldb
import datetime
import re
import configparser
#from helper import *
from IPython.display import clear_output
import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine

config = configparser.ConfigParser()
config.read('db.cfg')

clear_output()
timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%H:%M')
ts = datetime.datetime.now().strftime('%m-%d-%Y')

print 'Notebook run: {}'.format(timestamp)
print os.getcwd()

Notebook run: 2020-06-22_10:49
E:\RIS\Staff Folders\Samuel\Requests\SH\RIS DB


In [265]:
db = pysqldb.DbConnect(type='PG', server='dotdevrhpgsql01', database='ris')


Additional database connection details required:
User name (ris):soge
Password (ris)········
Database connection (PG) to ris on dotdevrhpgsql01 - user: soge 
Connection established 2020-06-22 10:49:29


In [266]:
table = 'SIGNAL_CONTROLLER'

In [27]:
#1. Identify IT table to be moved over to RIS DB
# RAW DATA INPUT FILES
# Copy '[GISGRID].[gisadmin].[SIGNAL_CONTROLLER]' to staging

def table_in_ris_db(dbo,table):    
    """
    Function to check if table is already in ris database
    :param dbo: pysqldb DbConnect instance of ris database
    :param table: table name
    :return: 
    """   
    
    tbl_names = db.dfquery("""SELECT table_name
                              FROM information_schema.tables
                              WHERE table_schema='staging'
                              AND table_type='BASE TABLE';""")
    
    
    if 'stg_' + table.lower() in list(tbl_names.table_name):
        return True
    else:
        return False

In [29]:
#table_in_ris_db(db,'SIGNAL_CONTROLLER')

In [ ]:
#2. Copy table to pg staging schema

def signal_data_to_stg():
    """
    Function to copy signals table to ris database
    :param dbo: pysqldb DbConnect instance of ris database
    :param table: table name
    :return: 
    """ 

    dest_pg = pysqldb.DbConnect(type='PG', server=SERVER, database=DATABASE)
    src_pg = pysqldb.DbConnect(server='dotgissql01', database='gisgrid', type='MS', user = 'GISUSER', password = 'GISUSER')
    src_schema = 'gisadmin'
    tbl = 'SIGNAL_CONTROLLER'
    copy_table_ms_to_pg(src_pg, src_schema, dest_pg, tbl)

In [ ]:
#3. Review log and decide if update is required.

def time_to_update(dbo, table, last_update, tbl_int):
    """
    Function to check if it is time to update passed table
    :param dbo: pysqldb DbConnect instance of ris database
    :param table: table name
    :param time: last date table was updated
    :param interval: table update interval
    :return: 
    """
    today = datetime.datetime.now()
    time_to_update =  today - last_update
    #if time_to_update > tbl_int:
    return True
    #else:
    return False

In [ ]:
   
#4. Clean table and move table to public
#   Clean up table and perform necessary transformations

def signal_data_clean(dbo, tbl):
    """
    Function to clean signals table after it has been moved to ris db
    :param dbo: pysqldb DbConnect instance of ris database
    :param table: table name
    :return: 
    """
    
    pwa_control()
    
    pysqldb.set_permissions(db, tbl, schema, src)

In [ ]:
#5. After table is successfully moved to public schema, run record_update to update record log 

def record_update(db, tbl):
    """
    Function to record that table has been updated
    :param dbo: pysqldb DbConnect instance of ris database
    :param table: table name
    :return: 
    """
    #TODO: Workflow to record the update details

In [ ]:
#6. Archive old data 

def archive_table(db, tbl):
    """
    Function to archive old data 
    :param dbo: pysqldb DbConnect instance of ris database
    :param table: table name
    :return: 
    """
    #TODO: Workflow to identify most recent data and archive old data
    
    
    pysqldb.pg_to_pg(src_pg, dest_pg, tbl, org_schema=src_schema, dest_schema='staging', dest_name=staging_name)
    
    src_pg = pysqldb.DbConnect(type='PG', server='dotdevrhpgsql01', database='ris')
    dest_pg = pysqldb.DbConnect(type='PG', server='dotdevrhpgsql01', database='ris')
    srch_schema = 'staging'
    table_name = tbl
    archive_table_pg(src_pg, src_schema, dest_pg, tbl)
    
signal_data_to_stg()

In [30]:
ts

'06-08-2020'

In [47]:
x = db.dfquery("""SELECT * 
              FROM public._tbl_updates_""")

x.last_updated.iloc[0]

In [52]:
table_name = 'ais_pd_core_f'

In [53]:
db.query("select last_updated from public._tbl_updates_ where table_name = '%s'" % table_name)

- Query run 2020-06-09 16:48:26.175000
 Query time: Query run in 4000 microseconds 
 * Returned 1 rows *


In [61]:
last_updated = db.data[0][0]

In [63]:
last_updated

datetime.datetime(2020, 5, 5, 15, 13, 36, 256261)

In [64]:
today = datetime.datetime.now()

In [68]:
today

datetime.datetime(2020, 6, 9, 16, 49, 35, 902000)

In [75]:
(today-last_updated).days

35

In [81]:
print """ALTER TABLE {tbl}
         RENAME TO archive_{tbl};""".format(tbl=tbl)

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 33))



NameError: name 'tbl' is not defined

In [84]:
table = 'stg_signal_controller'

In [86]:
print """CREATE TABLE public.{t}_copy AS SELECT * FROM staging.{t}""".format(t=table)

CREATE TABLE public.stg_signal_controller_copy AS SELECT * FROM staging.stg_signal_controller


In [97]:
db.connect()

Database connection (PG) to ris on dotdevrhpgsql01 - user: soge 
Connection established 2020-06-10 16:58:41


In [102]:
sig = db.dfquery("select * from staging.stg_signal_controller limit 1")

In [103]:
sig

,objectid,psgm_id,police,policekey,boro,geo1,geo2,seqno,gisgrid_gisadmin_sig_trpsgm_are,contrno,...,atcs_cabinet_address_hex,contractor_reference_number,longitude,latitude,signalinstalldate,point_x,point_y,last_updated,url,shape
0,7923815,10001,10005000580,I,1,11610,14010,1,001,1,...,0F96,10001,-73.99247600,40.71507200,1970-05-22 00:00:00.0000000,986336.14903097,199793.60932080,6/10/2020,<a href='https://www.google.com/maps/@40.71507...,010100000080BA494CC0192E4100AED8DF8C630841


In [107]:
x = list(sig.columns)

In [117]:
x

['objectid',
 'psgm_id',
 'police',
 'policekey',
 'boro',
 'geo1',
 'geo2',
 'seqno',
 'gisgrid_gisadmin_sig_trpsgm_are',
 'contrno',
 'contrtype',
 'contrsn',
 'rcotype',
 'rco_sn',
 'computer',
 'st1_code',
 'st1_name',
 'st2_code',
 'st2_name',
 'st3_code',
 'st3_name',
 'st4_code',
 'st4_name',
 'geokey1',
 'on_line',
 'mach',
 'sect',
 'logical',
 'cntrlpm',
 'tsmp',
 'psgm',
 'normalizedtype',
 'nodeid',
 'segmentid',
 'atcs_cabinet_address_hex',
 'contractor_reference_number',
 'longitude',
 'latitude',
 'signalinstalldate',
 'point_x',
 'point_y',
 'last_updated',
 'url',
 'shape']

In [ ]:
psgm_id,signalinstalldate,point_x,point_y,shape

In [ ]:
#Sig id, date installed, xy coordinates, nodeid, segmentid if its a mid block signal

In [13]:
db.query("""
            SELECT 'SIG' typ, psgm_id::int, longitude lon, latitude lat, signalinstalldate
            FROM staging.stg_signal_controller
            WHERE NormalizedType != 'Z'
            and longitude is not null

                    """)

- Query run 2020-06-15 11:35:15.105000
 Query time: Query run in 100000 microseconds 
 * Returned 13403 rows *


In [14]:
result = db.data

In [267]:
def xy_to_geom(cdb,*args): 
    
    """
    Function that generates the hexidecimal geomteric string 
    of every input coordinate. 
   
    Args:
        cdb param: Database Connection Function for PostgreSQL 
        
    *args
        xy arg: X,Y coordinates 
                format: [[x,y][x,y]] or xy=[x,y] or input x=x,y=y

    Returns:
        geom (str): Hexidecimal geomteric string 
        
    
    e.g.
    
    Input:
    
    xy_to_geom(cdb,[[1000245.0, 202806.0],[1000537.9999999995, 186726.9999999482],[1002334.9999999987, 250597.99999993874]])
    
    Returns:
    
    ['0101000020D7080000000000006A862E4100000000B0C10841',
     '0101000020D708000000000000B4882E410000000038CB0641']
    """
    
   
    
    try:
        
        if any(isinstance(item, list) for item in args[0]):  #args is a tuple 
            print('TRUE')
            count=0
            geom =[None]*len(args[0])
            for i in args[0]:
                db.query("select ST_GeomFROMText('POINT(' || {x}::text || ' ' ||  {y}::text || ')', 2263) geom ".format(x=i[0],y=i[1]))
                #print(g)
                geom[count]=db.data[0][0]
                count+=1

        else:
            #print(args[0])
            if isinstance(args[0], list):
                db.query("select ST_GeomFROMText('POINT(' || {x}::text || ' ' ||  {y}::text || ')', 2263) geom".format(x=args[0][0],y=args[0][1]))
                geom = db.data[0][0]

    except:
    
        xy = list(args)
        #print(lon_lat)
        db.query("select ST_GeomFROMText('POINT(' || {x}::text || ' ' ||  {y}::text || ')', 2263) geom".format(x=xy[0],y=xy[1]))
        geom = db.data[0][0]

    return geom

In [268]:
def lon_lat_to_geom(cdb,*args): #input either list of list [[lon,lat][lon,lat]] or list lon_lat=[lon,lat] or input lon=lon,lat=lat 
    
    """
    Function that generates the hexidecimal geomteric string 
    of every input coordinate. 
   
    Args:
        cdb param: Database Connection Function for PostgreSQL 
        
    *args
        lon_lat arg (float): Longitude,Latitude coordinates 
                             format: [[lon,lat][lon,lat]] or [lon,lat] or input lon=lon,lat=lat

    Returns:
        geom (str): Hexidecimal geomteric string 
        
    
    e.g.
    
    Input:
    
    lon_lat_to_geom(cdb,[[-73.99247600,40.71507200],[-73.99247600,40.71507200]])
    
    Returns:
    
    ['0101000020D7080000D5011E94BF192E41347ACDE4A4630841',
     '0101000020D7080000D5011E94BF192E41347ACDE4A4630841']
    """
    
    
  
   
    try:
        if any(isinstance(item, list) for item in args[0]): 
            #print('TRUE')
            count=0
            geom =[None]*len(args[0])
            for i in args[0]:
                #print(i)
                db.query("select st_transform(ST_PointFromText('POINT(' || {lon} || ' ' || {lat} || ')', 4326), 2263) geom ".format(lon=i[0],lat=i[1]))
                #print(g)
                geom[count]=db.data[0][0]
                count+=1

        else:
            #print(args[0])
            if isinstance(args[0], list):
                db.query("select st_transform(ST_PointFromText('POINT(' || {lon} || ' ' || {lat} || ')', 4326), 2263) geom ".format(lon=args[0][0],lat=args[0][1]))
                geom = db.data[0][0]

    except:
        lon_lat = list(args)
        db.query("select st_transform(ST_PointFromText('POINT(' || {lon} || ' ' || {lat} || ')', 4326), 2263) geom ".format(lon=lon_lat[0],lat=lon_lat[1]))
        geom = db.data[0][0]

    return geom

In [269]:
def to_itx(cdb,**args): 
    
    """
    Function that retrieves nearest intersection from the location
    of input control signal
   
    Args:
        cdb param: Database Connection Function for PostgreSQL
        
    *args:
        rad (int, optional): Search radius for locating nearest node from all way stop
        xy arg (optional if geom given): X,Y coordinates of all way stop
                                         format: [[x,y][x,y]] or xy=[x,y] or input x=x,y=y
        lon_lat arg (optional if geom given): Longitude,Latitude coordinates 
                                              format: [[lon,lat][lon,lat]] or [lon,lat] or input lon=lon, lat=lat
        geom arg (str, optional if xy or lon_lat given): Hexidecimal geomteric string X,Y coordinates of all way stop
        
    Returns:
        result (dataFrame): Nodeid, Distance between Nodeid and Control Signal, Nodeid Geometry, Control geometry, Coordinates
        val (int): 0 if no args given`    
        
    e.g.
    
    Input:
    
    to_itx(cdb,xy=[1000537.9999999995, 186726.9999999482],geom='0101000020D708000000000000B4882E410000000038CB0641')
    
    Returns:
    
    nodeid  distance    node_geom                   control_geom                xy
    26723   0.0340567   0101000020D7080000009...    0101000020D7080000000...    [1000538.0, 186727.0]
    """  
    
    db= cdb
    rad = args.get('rad', 50)  
    geom = args.get('geom', 0)
    xy = args.get('xy', 0)
    lon_lat= args.get('lon_lat', 0)
    coor = lambda x: map(float,x.replace('[','').replace(']','').split(','))  
    
         
    if len(args)==0:
            return 0
    
    if geom==0:
        if xy:
            geom = xy_to_geom(db,xy)
        else:
            geom = lon_lat_to_geom(db,lon_lat)

    
    if xy:
        
        query = """select nodeid,  masterid, st_distance(geom, '{geom}'::geometry) as distance,
                            geom as node_geom, '{geom}'::geometry as control_geom,
                            '{xy}' as xy,  ST_AsText(st_transform(geom::text,4326)) node_lon_lat,
                            ST_AsText(st_transform('{geom}'::text,4326)) control_lon_lat
               from node where st_dwithin('{geom}'::geometry,geom, {rad})
                        and is_int = true
               order by st_distance(geom, '{geom}'::geometry) ASC limit 1
               """.format(geom=geom, rad=rad, xy=xy)
        
        db.query(query)
        x=list(db.data[0:7])
        x.append(coor(r[4]))
        
        data=x
        #data = pd.DataFrame(x,index=['nodeid','distance','node_geom','control_geom','xy']).T
        
    elif lon_lat:

        query = """
                   select *, ST_AsText(st_transform(node_geom::text,4326)) node_lon_lat, ST_AsText(st_transform('{geom}'::text,4326)) control_lon_lat
                   from(
                   select nodeid,  masterid, st_distance(geom, '{geom}'::geometry) as distance,
                                geom as node_geom,
                                '{geom}'::geometry as sig_geom  
                   from node where st_dwithin('{geom}'::geometry,geom, {rad})
                            and is_int = true 
                   order by st_distance(geom, '{geom}'::geometry) ASC limit 1) x 
                   """.format(geom=geom, rad=rad, lon_lat=lon_lat)

        db.query(query)
        x=list(db.data[0:7])
        #x.append(coor(r[4]))

        data=x
        #data = pd.DataFrame(x,index=['nodeid','distance','node_geom','control_geom','lon_lat']).T     
            
    else:
        
        query = """select nodeid, masterid, st_distance(geom, '{geom}'::geometry) as distance,
                                geom as node_geom, '{geom}'::geometry as control_geom,
                                ST_AsText(st_transform(geom::text,4326)) node_lon_lat,
                                ST_AsText(st_transform('{geom}'::text,4326)) control_lon_lat
                   from node where st_dwithin('{geom}'::geometry,geom, {rad})
                            and is_int = true
                   order by st_distance(geom, '{geom}'::geometry) ASC limit 1
                   """.format(geom=geom, rad=rad)

        db.query(query)  
        r = list(db.data)
        data = pd.DataFrame(r,index=['nodeid','distance','node_geom','control_geom', 'node_lon_lat', 'control_lon_lat']).T
    


    return data

In [273]:
s2 = to_itx(db,rad=50,lon_lat=[float(-73.99247600),float(40.71507200)])      

- Query run 2020-06-22 10:55:27.339000
 Query time: Query run in 44000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-22 10:55:27.397000
 Query time: Query run in 33000 microseconds 
 * Returned 1 rows *


In [ ]:
-73.99247600 40.71507200

In [297]:
sig_data = pd.DataFrame()
failed_data = pd.DataFrame()

for i in result[:5]:
    try:
        s1 = [i[0],i[1]]
        s2 = to_itx(db,rad=50,lon_lat=[float(i[2]),float(i[3])]) 
        s1.extend(list(s2[0]))
        s1.append(i[4])
        print(s1)
        sig_data = sig_data.append(pd.DataFrame(s1,['control_type','control_id','nodeid','masterid','distance','node_geom',
                                                    'control_geom','node_lon_lat','control_lon_lat']).T, ignore_index = True)
    
        #print sig_data
    except:
        f = [i[0],i[1]]
        f1=lon_lat_to_geom(db,[float(i[2]),float(i[3])])
        f.extend([f1])
        failed_data=failed_data.append(pd.DataFrame(f,index = ['control_type','control_id','control_geom']).T,ignore_index=True)

sig_data['node_lon_lat'] = map(lambda x: map(float,x.strip('POINT()').split(' ')), sig_data['node_lon_lat'])
sig_data['control_lon_lat'] = map(lambda x: map(float,x.strip('POINT()').split(' ')), sig_data['control_lon_lat'])
sig_data = sig_data.reindex(columns=['nodeid', 'masterid','control_type', 'control_id', 'node_geom', 'node_lon_lat', 'control_geom', 'control_lon_lat', 'distance'])
    
print [sig_data, failed_data]


- Query run 2020-06-24 11:42:23.155000
 Query time: Query run in 46000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-24 11:42:23.242000
 Query time: Query run in 59000 microseconds 
 * Returned 1 rows *
['SIG', 10001, Decimal('92892'), 92892, 5.06652842691356, '0101000020D708000000853B78B7192E41009A2A33BD630841', '0101000020D7080000D5011E94BF192E41347ACDE4A4630841', 'POINT(-73.9924906245908 40.7150803402618)', 'POINT(-73.992476 40.7150719999998)', '1970-05-22 00:00:00.0000000']
- Query run 2020-06-24 11:42:23.307000
 Query time: Query run in 45000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-24 11:42:23.369000
 Query time: Query run in 36000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-24 11:42:23.428000
 Query time: Query run in 34000 microseconds 
 * Returned 1 rows *
['SIG', 10002, Decimal('20243'), 20245, 15.4097591900998, '0101000020D708000080BECDD3F5FD2D41002AB57F67AF0841', '0101000020D7080000F253F6F0E1FD2D416963385109AF0841', 'POINT(-74.0053071317689

KeyError: 'node_lon_lat'

In [156]:
failed_data

,control_type,control_id,contol_geom
0,SIG,10010,0101000020D708000032749E8FC0EC2D417A66556E00EC...
1,SIG,10035,0101000020D70800003E1A582A1AF02D41AF5A5E795710...
2,SIG,10042,0101000020D70800007F0C218AD9FF2D414CFE7E9CDF47...
3,SIG,10046,0101000020D7080000AA74562ACB062E41B601A46D4872...
4,SIG,10103,0101000020D70800003E9F3EA598022E4135019E43A55F...
5,SIG,10107,0101000020D708000017E7AB2F7B302E41B94928811651...
6,SIG,10138,0101000020D70800009625DAF2A7022E412F94756A30FF...
7,SIG,10160,0101000020D70800002D9E593B0AF32D415AD1101B41DD...
8,SIG,10161,0101000020D70800005FD23925EF002E41B3263B17500A...
9,SIG,10182,0101000020D70800009E2133F23BE72D410DE79F10FD06...


In [195]:
fd = failed_data.copy()
fd.values.tolist()

In [191]:
for i in (failed_data[['control_id','contol_geom']][:5]): print i

AttributeError: 'DataFrame' object has no attribute 'rows'

In [ ]:
 to_segment(db,**args)

In [160]:
failed_data.contol_geom[0]

'0101000020D708000032749E8FC0EC2D417A66556E00EC0741'

In [165]:
db.connect()

Database connection (PG) to ris on dotdevrhpgsql01 - user: soge 
Connection established 2020-06-15 17:32:48


In [278]:
def to_segment(db,**args): 


    """
    Function that retrieves nearest segment from the location
    of input control signal
   
    Args:
        db param: Database Connection Function for PostgreSQL
        
    *args:
        rad (int, optional): Search radius for locating nearest node from all way stop
        xy arg (optional if geom given): X,Y coordinates of all way stop
                                         format: [[x,y][x,y]] or xy=[x,y] or input x=x,y=y
        lon_lat arg (optional if geom given): Longitude,Latitude coordinates 
                                              format: [[lon,lat][lon,lat]] or [lon,lat] or input lon=lon, lat=lat
        geom arg (str, optional if xy or lon_lat given): Hexidecimal geomteric string X,Y coordinates of all way stop
        
    Returns:
        result (dataFrame): Nodeid, Distance between Nodeid and Control Signal, Nodeid Geometry, Control geometry, Coordinates
        val (int): 0 if no args given`    
        
    e.g.
    
    Input:
    
    to_itx(db,xy=[1000537.9999999995, 186726.9999999482],geom='0101000020D708000000000000B4882E410000000038CB0641')
    
    Returns:
    
    segmentid  distance    segment_geom                   control_geom                xy
    26723      0.0340567   0101000020D7080000009...       0101000020D7080000000...    [1000538.0, 186727.0]
    """  
    
    rad = args.get('rad', 50)  
    geom = args.get('geom', 0)
    xy = args.get('xy', 0)
    lon_lat= args.get('lon_lat', 0)
    coor = lambda x: map(float,x.replace('[','').replace(']','').split(','))  
    
         
    if len(args)==0:
            return 0
    
    if geom==0:
        if xy:
            geom = xy_to_geom(db,xy)
        else:
            geom = lon_lat_to_geom(db,lon_lat)

    
    if xy:
        
        query = """select segmentid,  mft, st_distance(geom, '{geom}'::geometry) as distance,
                            geom as segment_geom, '{geom}'::geometry as control_geom,
                            '{xy}' as xy,  ST_AsText(st_transform(geom::text,4326)) segment_lon_lat,
                            ST_AsText(st_transform('{geom}'::text,4326)) control_lon_lat
               from lion where st_dwithin('{geom}'::geometry,geom, {rad})
                        and is_int = true
               order by st_distance(geom, '{geom}'::geometry) ASC limit 1
               """.format(geom=geom, rad=rad, xy=xy)
        
        db.query(query)
        r = list(db.data[0])
        x.append(coor(r[4]))
        
        data=x
        #data = pd.DataFrame(x,index=['segmentid','distance','segment_geom','control_geom','xy']).T
        
    elif lon_lat:

        query = """
                   select *, ST_AsText(st_transform(segment_geom::text,4326)) segment_lon_lat, ST_AsText(st_transform('{geom}'::text,4326)) control_lon_lat
                   from(
                   select segmentid,  mft, st_distance(geom, '{geom}'::geometry) as distance,
                                geom as segment_geom,
                                '{geom}'::geometry as sig_geom  
                   from lion where st_dwithin('{geom}'::geometry,geom, {rad})
                   order by st_distance(geom, '{geom}'::geometry) ASC limit 1) x 
                   """.format(geom=geom, rad=rad, lon_lat=lon_lat)

        db.query(query)
        r = list(db.data[0])
        x.append(coor(r[4]))

        data=x
        #data = pd.DataFrame(x,index=['segmentid','distance','segment_geom','control_geom','lon_lat']).T     
            
    else:
        
        query = """select segmentid, mft, st_distance(geom, '{geom}'::geometry) as distance,
                                geom as node_geom, '{geom}'::geometry as control_geom,
                                ST_AsText(st_transform(geom::text,4326)) semgent_lon_lat,
                                ST_AsText(st_transform('{geom}'::text,4326)) control_lon_lat
                   from node where st_dwithin('{geom}'::geometry,geom, {rad})
                            and is_int = true
                   order by st_distance(geom, '{geom}'::geometry) ASC limit 1
                   """.format(geom=geom, rad=rad)
            
        db.query(query)
        r = list(db.data[0])
        data = pd.DataFrame(r,index=['segmentid','distance','segment_geom','control_geom', 'segment_lon_lat', 'control_lon_lat']).T
    


    return data


In [279]:
def to_segment(db,**args): 


    """
    Function that retrieves nearest segment from the location
    of input control signal
   
    Args:
        db param: Database Connection Function for PostgreSQL
        
    *args:
        rad (int, optional): Search radius for locating nearest node from all way stop
        xy arg (optional if geom given): X,Y coordinates of all way stop
                                         format: [[x,y][x,y]] or xy=[x,y] or input x=x,y=y
        lon_lat arg (optional if geom given): Longitude,Latitude coordinates 
                                              format: [[lon,lat][lon,lat]] or [lon,lat] or input lon=lon, lat=lat
        geom arg (str, optional if xy or lon_lat given): Hexidecimal geomteric string X,Y coordinates of all way stop
        
    Returns:
        result (dataFrame): Nodeid, Distance between Nodeid and Control Signal, Nodeid Geometry, Control geometry, Coordinates
        val (int): 0 if no args given`    
        
    e.g.
    
    Input:
    
    to_itx(db,xy=[1000537.9999999995, 186726.9999999482],geom='0101000020D708000000000000B4882E410000000038CB0641')
    
    Returns:
    
    segmentid  distance    segment_geom                   control_geom                xy
    26723      0.0340567   0101000020D7080000009...       0101000020D7080000000...    [1000538.0, 186727.0]
    """  
    
    rad = args.get('rad', 50)  
    geom = args.get('geom', 0)
    xy = args.get('xy', 0)
    lon_lat= args.get('lon_lat', 0)
    coor = lambda x: map(float,x.replace('[','').replace(']','').split(','))  
    
         
    if len(args)==0:
            return 0
    
    else:
        
        query = """select segmentid, mft, st_distance(geom, '{geom}'::geometry) as distance,
                    geom as segment_geom, '{geom}'::geometry as control_geom,
                    ST_AsText(st_transform(geom::text,4326)) semgent_lon_lat,
                    ST_AsText(st_transform('{geom}'::text,4326)) control_lon_lat
                   from lion where st_dwithin('{geom}'::geometry,geom, 50)               
                   order by st_distance(geom, '{geom}'::geometry) ASC limit 1
                   """.format(geom=geom, rad=rad)
            
        db.query(query)
        r = list(db.data[0])
        #data = pd.DataFrame(r,index=['segmentid','mft','distance','segment_geom','control_geom', 'segment_lon_lat', 'control_lon_lat']).T
    


    return r

In [207]:
fd = failed_data.copy()
fd = fd.values.tolist()

In [211]:
fd[:5]

[['SIG', 10010, '0101000020D708000032749E8FC0EC2D417A66556E00EC0741'],
 ['SIG', 10035, '0101000020D70800003E1A582A1AF02D41AF5A5E7957100841'],
 ['SIG', 10042, '0101000020D70800007F0C218AD9FF2D414CFE7E9CDF470841'],
 ['SIG', 10046, '0101000020D7080000AA74562ACB062E41B601A46D48720841'],
 ['SIG', 10103, '0101000020D70800003E9F3EA598022E4135019E43A55F0841']]

In [214]:
db.connect()

Database connection (PG) to ris on dotdevrhpgsql01 - user: soge 
Connection established 2020-06-16 11:48:21


In [243]:
fd_segs = pd.DataFrame()

for i in fd:
    
    try:
        s1 = [i[0],i[1]]
        s2 = to_segment(db,rad=50,geom=i[2])
        s1.extend(list(s2))
        

        fd_segs = fd_segs.append(pd.DataFrame(s1,['control_type','control_id','segmentid','mft','distance','segment_geom','control_geom', 'segment_lon_lat', 'control_lon_lat']).T, ignore_index = True)
    
    except:
        print(i) 

- Query run 2020-06-16 14:13:06.260000
 Query time: Query run in 42000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:06.319000
 Query time: Query run in 32000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:06.386000
 Query time: Query run in 40000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:06.453000
 Query time: Query run in 39000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:06.525000
 Query time: Query run in 46000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:06.585000
 Query time: Query run in 35000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:06.653000
 Query time: Query run in 41000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:06.713000
 Query time: Query run in 34000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:06.784000
 Query time: Query run in 42000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:06.841000
 Query 

 * Returned 1 rows *
- Query run 2020-06-16 14:13:11.312000
 Query time: Query run in 34000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:11.381000
 Query time: Query run in 41000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:11.450000
 Query time: Query run in 41000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:11.513000
 Query time: Query run in 38000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:11.575000
 Query time: Query run in 36000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:11.641000
 Query time: Query run in 40000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:11.705000
 Query time: Query run in 35000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:11.771000
 Query time: Query run in 39000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:11.833000
 Query time: Query run in 37000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14

 * Returned 1 rows *
- Query run 2020-06-16 14:13:16.516000
 Query time: Query run in 36000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:16.583000
 Query time: Query run in 39000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:16.645000
 Query time: Query run in 37000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:16.705000
 Query time: Query run in 34000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:16.767000
 Query time: Query run in 39000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:16.831000
 Query time: Query run in 37000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:16.891000
 Query time: Query run in 34000 microseconds 
 * Returned 0 rows *
['SIG', 12916, '0101000020D70800005019293A3F1F2E41F65CFAB4059A0941']
- Query run 2020-06-16 14:13:16.957000
 Query time: Query run in 36000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:17.016000
 Query time: Query run 

 * Returned 1 rows *
- Query run 2020-06-16 14:13:21.514000
 Query time: Query run in 40000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:21.585000
 Query time: Query run in 46000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:21.658000
 Query time: Query run in 46000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:21.739000
 Query time: Query run in 56000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:21.802000
 Query time: Query run in 38000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:21.862000
 Query time: Query run in 41000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:21.936000
 Query time: Query run in 49000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:22.006000
 Query time: Query run in 43000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:22.069000
 Query time: Query run in 38000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14

 * Returned 1 rows *
- Query run 2020-06-16 14:13:26.454000
 Query time: Query run in 38000 microseconds 
 * Returned 0 rows *
['SIG', 21981, '0101000020D70800001480AD27BF472F4196C8A13A02840E41']
- Query run 2020-06-16 14:13:26.517000
 Query time: Query run in 36000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:26.595000
 Query time: Query run in 52000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:26.657000
 Query time: Query run in 35000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:26.727000
 Query time: Query run in 45000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:26.795000
 Query time: Query run in 40000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:26.863000
 Query time: Query run in 40000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:26.927000
 Query time: Query run in 36000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:26.988000
 Query time: Query run 

 * Returned 1 rows *
- Query run 2020-06-16 14:13:35.847000
 Query time: Query run in 39000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:35.914000
 Query time: Query run in 38000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:35.972000
 Query time: Query run in 34000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:36.032000
 Query time: Query run in 35000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:36.099000
 Query time: Query run in 40000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:36.167000
 Query time: Query run in 43000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:36.237000
 Query time: Query run in 45000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:36.299000
 Query time: Query run in 35000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:36.365000
 Query time: Query run in 39000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14

 * Returned 1 rows *
- Query run 2020-06-16 14:13:40.721000
 Query time: Query run in 46000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:40.785000
 Query time: Query run in 38000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:40.851000
 Query time: Query run in 41000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:40.912000
 Query time: Query run in 37000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:40.961000
 Query time: Query run in 34000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:41.019000
 Query time: Query run in 33000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:41.086000
 Query time: Query run in 41000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:41.145000
 Query time: Query run in 34000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14:13:41.219000
 Query time: Query run in 46000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 14

In [246]:
nn = failed_data.copy()
nn = nn.values.tolist()

sig_wsegs = pd.DataFrame()
failed= []

for i in nn:
    try:
        s3 = [i[0],i[1]]
        s4 = to_segment(db,rad=50,geom=i[2])
        s3.extend(list(s2))

        sig_wsegs = sig_wsegs.append(pd.DataFrame(s3,['control_type','control_id','segmentid','mft','distance','segment_geom','control_geom', 'segment_lon_lat', 'control_lon_lat']).T, ignore_index = True)

    except:
        failed_data.append(i)

- Query run 2020-06-16 17:07:15.179000
 Query time: Query run in 45000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:15.231000
 Query time: Query run in 34000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:15.298000
 Query time: Query run in 41000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:15.359000
 Query time: Query run in 34000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:15.417000
 Query time: Query run in 34000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:15.475000
 Query time: Query run in 33000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:15.539000
 Query time: Query run in 38000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:15.602000
 Query time: Query run in 38000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:15.663000
 Query time: Query run in 34000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:15.727000
 Query 

 * Returned 1 rows *
- Query run 2020-06-16 17:07:20.130000
 Query time: Query run in 39000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:20.192000
 Query time: Query run in 38000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:20.260000
 Query time: Query run in 39000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:20.322000
 Query time: Query run in 38000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:20.381000
 Query time: Query run in 33000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:20.446000
 Query time: Query run in 39000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:20.517000
 Query time: Query run in 45000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:20.577000
 Query time: Query run in 34000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:20.638000
 Query time: Query run in 37000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17

 * Returned 1 rows *
- Query run 2020-06-16 17:07:25.110000
 Query time: Query run in 35000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:25.174000
 Query time: Query run in 37000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:25.247000
 Query time: Query run in 45000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:25.297000
 Query time: Query run in 34000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:25.357000
 Query time: Query run in 35000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:25.421000
 Query time: Query run in 39000 microseconds 
 * Returned 0 rows *
- Query run 2020-06-16 17:07:25.490000
 Query time: Query run in 40000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:25.547000
 Query time: Query run in 33000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:25.602000
 Query time: Query run in 32000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17

 * Returned 1 rows *
- Query run 2020-06-16 17:07:29.980000
 Query time: Query run in 34000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:30.038000
 Query time: Query run in 33000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:30.096000
 Query time: Query run in 34000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:30.158000
 Query time: Query run in 36000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:30.222000
 Query time: Query run in 37000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:30.280000
 Query time: Query run in 34000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:30.339000
 Query time: Query run in 34000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:30.397000
 Query time: Query run in 34000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:30.462000
 Query time: Query run in 37000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17

 * Returned 1 rows *
- Query run 2020-06-16 17:07:34.842000
 Query time: Query run in 40000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:34.904000
 Query time: Query run in 35000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:34.971000
 Query time: Query run in 39000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:35.033000
 Query time: Query run in 37000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:35.085000
 Query time: Query run in 35000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:35.156000
 Query time: Query run in 45000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:35.215000
 Query time: Query run in 32000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:35.282000
 Query time: Query run in 40000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:35.345000
 Query time: Query run in 39000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17

 * Returned 1 rows *
- Query run 2020-06-16 17:07:39.928000
 Query time: Query run in 41000 microseconds 
 * Returned 0 rows *
- Query run 2020-06-16 17:07:39.997000
 Query time: Query run in 41000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:40.069000
 Query time: Query run in 41000 microseconds 
 * Returned 0 rows *
- Query run 2020-06-16 17:07:40.138000
 Query time: Query run in 41000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:40.208000
 Query time: Query run in 44000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:40.270000
 Query time: Query run in 39000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:40.339000
 Query time: Query run in 40000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:40.408000
 Query time: Query run in 42000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:40.474000
 Query time: Query run in 40000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17

 * Returned 1 rows *
- Query run 2020-06-16 17:07:44.860000
 Query time: Query run in 32000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:44.933000
 Query time: Query run in 49000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:44.996000
 Query time: Query run in 38000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:45.055000
 Query time: Query run in 34000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:45.118000
 Query time: Query run in 36000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:45.186000
 Query time: Query run in 42000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:45.243000
 Query time: Query run in 32000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:45.305000
 Query time: Query run in 36000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:45.371000
 Query time: Query run in 38000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17

 * Returned 1 rows *
- Query run 2020-06-16 17:07:49.720000
 Query time: Query run in 31000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:49.779000
 Query time: Query run in 32000 microseconds 
 * Returned 0 rows *
- Query run 2020-06-16 17:07:49.841000
 Query time: Query run in 36000 microseconds 
 * Returned 0 rows *
- Query run 2020-06-16 17:07:49.902000
 Query time: Query run in 36000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:49.964000
 Query time: Query run in 36000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:50.017000
 Query time: Query run in 36000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:50.078000
 Query time: Query run in 35000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:50.139000
 Query time: Query run in 35000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17:07:50.199000
 Query time: Query run in 35000 microseconds 
 * Returned 1 rows *
- Query run 2020-06-16 17

In [256]:
x = pd.concat([sig_data,sig_wsegs])

C:\Python27\ArcGIS10.4\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [260]:
x.reindex(columns=['control_type','control_id','nodeid', 'masterid', 'segmentid', 'mft', 'distance', 'control_lon_lat', 'node_lon_lat', 'segment_lon_lat' ,'control_geom', 'node_geom', 'segment_geom'])


,control_type,control_id,nodeid,masterid,segmentid,mft,distance,control_lon_lat,node_lon_lat,segment_lon_lat,control_geom,node_geom,segment_geom
0,SIG,10001,92892,92892,NaN,NaN,5.06653,"[-73.992476, 40.715072]","[-73.9924906246, 40.7150803403]",NaN,0101000020D7080000D5011E94BF192E41347ACDE4A463...,0101000020D708000000853B78B7192E41009A2A33BD63...,NaN
1,SIG,10002,20243,20245,NaN,NaN,15.4098,"[-74.005343, 40.721694]","[-74.0053071318, 40.7217263149]",NaN,0101000020D7080000F253F6F0E1FD2D416963385109AF...,0101000020D708000080BECDD3F5FD2D41002AB57F67AF...,NaN
2,SIG,10003,20216,20216,NaN,NaN,27.9408,"[-74.005212, 40.720552]","[-74.0052084874, 40.7206286442]",NaN,0101000020D708000083DE03842AFE2D41A4B226C608A2...,0101000020D708000000FB62772CFE2D410060082AE8A2...,NaN
3,SIG,10004,20215,20215,NaN,NaN,0.545153,"[-74.005188, 40.719838]","[-74.0051861626, 40.7198374664]",NaN,0101000020D7080000E5AD4DCA37FE2D412DEDC0B6E799...,0101000020D708000080B210CF38FE2D41003C8D28E699...,NaN
4,SIG,10005,20214,20214,NaN,NaN,0.397664,"[-74.005173, 40.719004]","[-74.0051742031, 40.7190045945]",NaN,0101000020D70800006FD2FF1140FE2D4100D7CFE56890...,0101000020D7080000003140673FFE2D41004A6CA16A90...,NaN
5,SIG,10006,20482,20482,NaN,NaN,0.599102,"[-73.999311, 40.717549]","[-73.9993124177, 40.7175502411]",NaN,0101000020D7080000190F0EFEF10A2E41D688B4C1D77F...,0101000020D70800008018D634F10A2E41001CC75FDB7F...,NaN
6,SIG,10007,9034094,15701,NaN,NaN,17.2067,"[-73.996833, 40.715098]","[-73.996865615, 40.7150578172]",NaN,0101000020D7080000993826E64F102E41702BB815F063...,0101000020D708000080D22FD13D102E410086BAF67A63...,NaN
7,SIG,10008,20569,78691,NaN,NaN,14.1486,"[-73.996079, 40.716196]","[-73.9960587278, 40.7162316399]",NaN,0101000020D708000031CC72E8F1112E415164746E7070...,0101000020D708000080387225FD112E4100A2A54FD870...,NaN
8,SIG,10009,15615,15615,NaN,NaN,23.563,"[-73.997486, 40.714006]","[-73.9975044541, 40.7140691319]",NaN,0101000020D70800006EF106E0E50E2E4182E64A3F8157...,0101000020D708000080905AA4DB0E2E4100B0CD403958...,NaN
9,SIG,10011,15234,15234,NaN,NaN,0.349685,"[-74.011549, 40.70495]","[-74.0115498107, 40.7049507352]",NaN,0101000020D70800002164CED76FF02D412031CBDC67F0...,0101000020D708000000CCBB646FF02D41006C70016AF0...,NaN


In [263]:
sig_data.columns

Index([u'nodeid', u'masterid', u'control_type', u'control_id', u'node_geom',
       u'node_lon_lat', u'control_geom', u'control_lon_lat', u'distance'],
      dtype='object')

In [280]:
query = """select segmentid, mft, st_distance(geom, '{geom}'::geometry) as distance,
            geom as segment_geom, '{geom}'::geometry as control_geom,
            ST_AsText(st_transform(geom::text,4326)) semgent_lon_lat,
            ST_AsText(st_transform('{geom}'::text,4326)) control_lon_lat
           from lion where st_dwithin('{geom}'::geometry,geom, 50)               
           order by st_distance(geom, '{geom}'::geometry) ASC limit 1
           """.format(geom='0101000020D708000032749E8FC0EC2D417A66556E00EC0741', rad=50)

db.query(query)
r = list(db.data[0])
#data = pd.DataFrame(r,index=['segmentid','distance','segment_geom','control_geom', 'segment_lon_lat', 'control_lon_lat']).T




- Query run 2020-06-24 11:26:15.992000
 Query time: Query run in 145000 microseconds 
 * Returned 1 rows *


In [290]:
x = list(db.data[0])

In [292]:
x.append('apple sauce')

In [293]:
x

['0023056',
 23056,
 5.7760019960064,
 '0105000020D70800000100000001020000000200000000345195B5EC2D4100E2E0BCE4EB074180A0AB6896EC2D4100B0CEE353F10741',
 '0101000020D708000032749E8FC0EC2D417A66556E00EC0741',
 'MULTILINESTRING((-74.0132697948119 40.7045534963101,-74.0133261085556 40.705030787933))',
 'POINT(-74.01325 40.7045629999998)',
 'apple sauce']

In [179]:
r

['0023056',
 23056,
 5.7760019960064,
 '0105000020D70800000100000001020000000200000000345195B5EC2D4100E2E0BCE4EB074180A0AB6896EC2D4100B0CEE353F10741',
 '0101000020D708000032749E8FC0EC2D417A66556E00EC0741',
 'MULTILINESTRING((-74.0132697948119 40.7045534963101,-74.0133261085556 40.705030787933))',
 'POINT(-74.01325 40.7045629999998)']

In [ ]:
select segmentid, mft, st_distance(geom, '{geom}'::geometry) as distance,
    geom as segment_geom, '{geom}'::geometry as control_geom,
    ST_AsText(st_transform(geom::text,4326)) semgent_lon_lat,
    ST_AsText(st_transform('{geom}'::text,4326)) control_lon_lat
from lion where st_dwithin('{geom}'::geometry,geom, 50)               
order by st_distance(geom, '{geom}'::geometry) ASC limit 1

In [ ]:
select segmentid, mft, st_distance(geom, '0101000020D708000032749E8FC0EC2D417A66556E00EC0741'::geometry) as distance,
	geom as segment_geom, '0101000020D708000032749E8FC0EC2D417A66556E00EC0741'::geometry as control_geom,
	ST_AsText(st_transform(geom::text,4326)) semgent_lon_lat,
	ST_AsText(st_transform('0101000020D708000032749E8FC0EC2D417A66556E00EC0741'::text,4326)) control_lon_lat
from lion where st_dwithin('0101000020D708000032749E8FC0EC2D417A66556E00EC0741'::geometry,geom, 50)               
order by st_distance(geom, '0101000020D708000032749E8FC0EC2D417A66556E00EC0741'::geometry) ASC limit 1

In [299]:
db.query("select last_updated from public._tbl_updates_ where table_name = '%s'" % 'signal_controller')


- Query run 2020-06-26 11:26:54.447000
 Query time: Query run in 88000 microseconds 
 * Returned 0 rows *


In [301]:
db.data

[]

In [310]:
datetime.datetime.today().day

26

In [ ]:
pysqldb.__version__()

In [315]:
from ris import pysqldb.__version__

SyntaxError: invalid syntax (<ipython-input-315-5ffb40d7b7e9>, line 1)

In [314]:
print pysqldb.__version__

AttributeError: 'module' object has no attribute '__version__'

In [330]:
db.query("select * from working._tbl_updates_ where table_name = '%s'" % 'signal_controller')
        

- Query run 2020-06-29 11:35:58.228000
 Query time: Query run in 3000 microseconds 
 * Returned 0 rows *


In [331]:
db.data[0]

IndexError: list index out of range

In [334]:
db.query("SELECT * FROM working._tbl_updates_;")


- Query run 2020-06-29 11:37:01.011000
 Query time: Query run in 3000 microseconds 
 * Returned 1 rows *


In [338]:
db.data[0]

('SIGNAL_CONTROLLER',
 datetime.datetime(2020, 6, 29, 11, 29, 36, 69758),
 1,
 datetime.datetime(2020, 6, 29, 11, 29, 36, 69758))

In [344]:
db.query(""" 

SELECT * FROM working._tbl_updates_ where table_name = '%s' 

""" % 'SIGNAL_CONTROLLER')


- Query run 2020-06-29 11:40:24.419000
 Query time: Query run in 3000 microseconds 
 * Returned 1 rows *


In [348]:
db.data[0][2]

1

In [350]:
db.my_tables()

,tablename,tableowner


In [352]:
print 'Cleanging staging.stg_%s' % 'table_name'

Cleanging staging.stg_table_name
